In [1]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100
rcParams["figure.figsize"] = 12, 4
rcParams["font.size"] = 16
rcParams["text.usetex"] = False
rcParams["font.family"] = ["sans-serif"]
rcParams["font.sans-serif"] = ["cmss10"]
rcParams["axes.unicode_minus"] = False

In [3]:
import numpy as np
import pymc3 as pm
import exoplanet as exo
import starry
import theano
import theano.tensor as tt

In [46]:
# Define all arguments
kwargs = {
    "u":        [0.4, 0.26],
    "b":        0.15,
    "zo":       1.0,
    "ro":       0.1
}
theano_kwargs = {}
for key in kwargs.keys():
    theano_kwargs[key] = theano.shared(np.float64(kwargs[key]), name=key)

# Compute the rv and its gradient using starry
map = starry.Map(ydeg=0, udeg=2)
map[1:] = kwargs.pop("u")
rv, grad = map.flux(gradient=True, **kwargs)

# Instantiate the theano op
model = map.flux_op(**theano_kwargs)

# Compute the gradient using Theano
varnames = sorted(theano_kwargs.keys())
vars = [theano_kwargs[k] for k in varnames]
computed = dict(zip(varnames,
                    theano.function([], theano.grad(model[0], vars))()))

# Compare
for key in theano_kwargs.keys():
    if key == "zo":
        continue
    print(key, np.squeeze(grad[key]), np.squeeze(computed[key]))
    assert np.allclose(np.squeeze(grad[key]), np.squeeze(computed[key]))

u [-0.00472115 -0.00244178] [-0.00472115 -0.00244178]
b 0.0007568178400598446 0.0007568178400598446
ro -0.24129282382813103 -0.24129282382813103
